#### 문제 설명  
사칙연산에서 더하기(+)는 결합법칙이 성립하지만, 빼기(-)는 결합법칙이 성립하지 않습니다.  
예를 들어 식 1 - 5 - 3은 연산 순서에 따라 다음과 같이 다른 결과를 가집니다.  
  
- ((1 - 5) - 3) = -7
- (1 - (5 - 3)) = -1  
  
위 예시와 같이 뺄셈은 연산 순서에 따라 그 결과가 바뀔 수 있습니다.  
또 다른 예로 식 1 - 3 + 5 - 8은 연산 순서에 따라 다음과 같이 5가지 결과가 나옵니다.  
  
- (((1 - 3) + 5) - 8) = -5
- ((1 - (3 + 5)) - 8) = -15
- (1 - ((3 + 5) - 8)) = 1
- (1 - (3 + (5 - 8))) = 1
- ((1 - 3) + (5 - 8)) = -5  
  
위와 같이 서로 다른 연산 순서의 계산 결과는 [-15, -5, -5, 1, 1]이 되며, 이중 최댓값은 1입니다.
문자열 형태의 숫자와, 더하기 기호(+), 뺄셈 기호(-)가 들어있는 배열 arr가 매개변수로 주어질 때, 서로 다른 연산순서의 계산 결과 중 최댓값을 return 하도록 solution 함수를 완성해 주세요.  
 
#### 제한 조건  
- arr는 두 연산자 +, - 와 숫자가 들어있는 배열이며, 길이는 3 이상 201 이하 입니다.
    - arr의 길이는 항상 홀수입니다.
    - arr에 들어있는 숫자의 개수는 2개 이상 101개 이하이며, 연산자의 개수는 (숫자의 개수) -1 입니다.
    - 숫자는 1 이상 1,000 이하의 자연수가 문자열 형태로 들어있습니다.. (ex : 456)
- 배열의 첫 번째 원소와 마지막 원소는 반드시 숫자이며, 숫자와 연산자가 항상 번갈아가며 들어있습니다.
  
#### 입출력 예  
|arr|	result|
|:---:|:---:|
|["1", "-", "3", "+", "5", "-", "8"]|	1|
|["5", "-", "3", "+", "1", "+", "2", "-", "4"]|	3|  

---

### 문제 풀이

In [1]:
def max_calculate(arr, len_num, max_dp, min_dp, start, end): # [start, end] 구간을 연산한 최댓값 반환
    if max_dp[start][end] != -1000*101: # 이미 구한 값이라면 바로 반환
        return max_dp[start][end]
    for i in range(start, end):
        # [start, i], [i+1, end] 구간에 대해서
        if arr[2*i+1] == '+': # [start, i] + [i+1, end] 일 때
            max_dp[start][end] = max(max_dp[start][end], max_calculate(arr, len_num, max_dp, min_dp, start, i) + max_calculate(arr, len_num, max_dp, min_dp, i+1, end))
            # [start, end] 구간의 연산 최댓값 = [start, i] 구간의 연산 최댓값 + [i+1, end] 구간의 연산 최댓값
        else: # arr[2*i+1] == '-'  # [start, i] - [i+1, end] 일 때
            max_dp[start][end] = max(max_dp[start][end], max_calculate(arr, len_num, max_dp, min_dp, start, i) - min_calculate(arr, len_num, max_dp, min_dp, i+1, end)) 
            # [start, end] 구간의 연산 최댓값 = [start, i] 구간의 연산 최댓값 - [i+1, end] 구간의 연산 최솟값
    return max_dp[start][end] # [start, end] 구간을 연산한 최댓값 반환

def min_calculate(arr, len_num, max_dp, min_dp, start, end): # [start, end] 구간을 연산한 최솟값 반환
    if min_dp[start][end] != 1000*101: # 이미 구한 값이라면 바로 반환
        return min_dp[start][end]
    for i in range(start, end):
        # [start, i], [i+1, end] 구간에 대해서
        if arr[2*i+1] == '+': # [start, i] + [i+1, end] 일 때
            min_dp[start][end] = min(min_dp[start][end], min_calculate(arr, len_num, max_dp, min_dp, start, i) + min_calculate(arr, len_num, max_dp, min_dp, i+1, end))    
            # [start, end] 구간의 연산 최솟값 = [start, i] 구간의 연산 최솟값 + [i+1, end] 구간의 연산 최솟값
        else: # arr[2*i+1] == '-': # [start, i] - [i+1, end] 일 때
            min_dp[start][end] = min(min_dp[start][end], min_calculate(arr, len_num, max_dp, min_dp, start, i) - max_calculate(arr, len_num, max_dp, min_dp, i+1, end))
            # [start, end] 구간의 연산 최솟값 = [start, i] 구간의 연산 최솟값 - [i+1, end] 구간의 연산 최댓값
    return min_dp[start][end] # [start, end] 구간을 연산한 최솟값 반환
    
        
def solution(arr):
    len_num = (len(arr)//2)+1 # 숫자 개수
    answer = 0 # 답
    max_dp = [[-1000*101 for _ in range(len_num)] for _ in range(len_num)] # 구간 범위 간 사칙연산 최댓값
    min_dp = [[1000*101 for _ in range(len_num)] for _ in range(len_num)] # 구간 범위 간 사칙연산 최솟값
    for i in range(len_num):
        max_dp[i][i] = int(arr[2*i])
        min_dp[i][i] = int(arr[2*i])
        # [i, i] 구간 연산의 최댓값, 최솟값은 i번째 숫자 값으로 초기화
    answer = max_calculate(arr, len_num, max_dp, min_dp, 0, len_num-1) # 모든 숫자 연산에 대한 최댓값
    return answer # 답 반환

In [2]:
arr = ["1", "-", "3", "+", "5", "-", "8"]
solution(arr)

1

In [3]:
arr = ["5", "-", "3", "+", "1", "+", "2", "-", "4"]
solution(arr)

3